# 3. I/O handling (reproducible experiments, snapshotting)

In [7]:
import numpy as np
import torch

In [8]:
# set logger and enforce reproducibility
from GPErks.log.logger import get_logger
from GPErks.utils.random import set_seed
log = get_logger()
seed = 8
set_seed(seed)  # reproducible sampling


<br/>

**2D function example**: Currin et al. (1988)

$f(x) = \left[1 - \exp{\left(1 - \dfrac{1}{2 x_2}\right)}\right]\,\left(\dfrac{2300 x_{1}^3 + 1900 x_{1}^2 + 2092 x_{1} + 60}{100 x_{1}^3 + 500 x_{1}^2 + 4 x_{1} + 20}\right)$

<br/>


In [9]:
# function to learn (normally a high-dimensional, expensive deterministic model)
from GPErks.utils.test_functions import currin_exp
D = 2

In [10]:
# build dataset
from GPErks.gp.data.dataset import Dataset
dataset = Dataset.build_from_function(
    currin_exp,
    D,
    n_train_samples=20,
    n_test_samples=25,
    design="lhs",
    seed=seed,
)

In [11]:
# choose likelihood
from gpytorch.likelihoods import GaussianLikelihood
likelihood = GaussianLikelihood()

In [12]:
# choose mean function
from GPErks.gp.mean import LinearMean
mean_function = LinearMean(degree=1, input_size=dataset.input_size, bias=True)

In [13]:
# choose kernel
from gpytorch.kernels import RBFKernel, ScaleKernel
kernel = ScaleKernel(RBFKernel(ard_num_dims=dataset.input_size))

In [14]:
# choose metrics
from torchmetrics import MeanSquaredError, R2Score
metrics = [MeanSquaredError(), R2Score()]

In [15]:
# define experiment
from GPErks.gp.experiment import GPExperiment
experiment = GPExperiment(
    dataset,
    likelihood,
    mean_function,
    kernel,
    n_restarts=3,
    metrics=metrics,
    seed=seed  # reproducible training
)

In [16]:
# dump experiment in config file
config_file = "./example_3.ini"
experiment.save_to_config_file(config_file)

In [17]:
# load experiment from config file
del experiment  # let's delete the original experiment before being able to re-create it from file
from GPErks.gp.experiment import load_experiment_from_config_file
experiment = load_experiment_from_config_file(
    config_file,
    dataset  # data is not saved in config file to save memory, so we still need to pass the dataset used!
)

In [18]:
# choose training options: device + optimizer
device = "cpu"
optimizer = torch.optim.Adam(experiment.model.parameters(), lr=0.1)

In [19]:
# train model
from GPErks.train.emulator import GPEmulator
emulator = GPEmulator(experiment, device)

In [20]:
# snapshotting
from GPErks.serialization.path import posix_path
from GPErks.train.snapshot import (
    EveryEpochSnapshottingCriterion,
    EveryNEpochsSnapshottingCriterion,
    NeverSaveSnapshottingCriterion
)
import os

snapshot_dir = posix_path(os.getcwd(), "snapshot", "example_3")
train_restart_template = "restart_{restart}"
train_epoch_template = "epoch_{epoch}.pth"

snapshot_file = train_epoch_template
snpc = EveryEpochSnapshottingCriterion(
    posix_path(snapshot_dir, train_restart_template),
    snapshot_file
)

# training
emulator.train(optimizer, snapshotting_criterion=snpc)

INFO:2025-05-16 16:33:38,190:emulator:train:L82: Training emulator...
INFO:2025-05-16 16:33:38,190:emulator:train:L98: Running restart 1...
/Users/slongobardi/Projects/GPErks/.venv/lib/python3.12/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
INFO:2025-05-16 16:33:38,218:emulator:_train_once:L303: [  1/100] Training Loss: 0.8316 - MeanSquaredError: 0.0849 - R2Score: 0.9151
INFO:2025-05-16 16:33:38,221:emulator:_train_once:L303: [  2/100] Training Loss: 0.7894 - MeanSquaredError: 0.0816 - R2Score: 0.9184
INFO:2025-05-16 16:33:38,226:emulator:_train_once:L303: [  3/100] Training Loss: 0.7519 - MeanSquaredError: 0.0795 - R2Score: 0.9205
INFO:2025-05-16 16:33:38,230:emulator:_train_once:L303: [  4/100] Training Loss: 0.7177 - MeanSquaredError: 0.0780 - R2Score: 0.9220
INFO:2025-05-16 16:33:38,233:emulator:_train_once:L303: [  5/100] Training Loss: 0.6858 - MeanSquaredError: 0.

(OrderedDict([('likelihood.noise_covar.raw_noise', tensor([-3.6342])),
              ('likelihood.noise_covar.raw_noise_constraint.lower_bound',
               tensor(1.0000e-04)),
              ('likelihood.noise_covar.raw_noise_constraint.upper_bound',
               tensor(inf)),
              ('mean_module.weights',
               tensor([[-0.0557],
                       [-2.5851]])),
              ('mean_module.bias', tensor([1.2667])),
              ('covar_module.raw_outputscale', tensor(-2.0184)),
              ('covar_module.base_kernel.raw_lengthscale',
               tensor([[-2.0959,  2.1659]])),
              ('covar_module.base_kernel.raw_lengthscale_constraint.lower_bound',
               tensor(0.)),
              ('covar_module.base_kernel.raw_lengthscale_constraint.upper_bound',
               tensor(inf)),
              ('covar_module.raw_outputscale_constraint.lower_bound',
               tensor(0.)),
              ('covar_module.raw_outputscale_constraint.upper_bo

In [21]:
# inference on stored test set
from GPErks.perks.inference import Inference
inference = Inference(emulator)
inference.summary()

                   Score
MeanSquaredError  0.7254
R2Score           0.9013


In [22]:
# loading emulator
best_model_file = posix_path(
    snapshot_dir,
    "best_model.pth"
)
best_model_state = torch.load(best_model_file, map_location=torch.device(device))

emulator1 = GPEmulator(experiment, device)
emulator1.model.load_state_dict(best_model_state)

<All keys matched successfully>

In [23]:
inference = Inference(emulator1)
inference.summary()

                   Score
MeanSquaredError  0.7254
R2Score           0.9013
